In [13]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame

## Case Study 4: Duplicate Record Removal Pipeline

Objective: Identify and remove duplicate movie records based on movieId and title, saving clean data in Avro format.

Scenario: The movies.csv file in HDFS contains duplicate records that need to be cleaned.
Steps:

Ingestion: Load movies.csv into a Spark DataFrame from HDFS.

Transformation:

Use DataFrames to identify duplicates based on movieId and title.
Convert the DataFrame to an RDD to perform custom filtering operations using distinct() on a composite key (movieId, title).
Validation:

Count the number of duplicates removed by comparing the record counts before and after transformation.
Storage:

Save the cleaned data as Avro files in GCP Cloud Storage.

In [2]:
// Step 1: Initialize SparkSession
val spark = SparkSession.builder()
  .appName("Transfer File from GCS to HDFS")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@477e84e0


org.apache.spark.sql.SparkSession@477e84e0

In [5]:
// Step 2: Define the GCS path and HDFS path
val bucketName = "scala_assgn_bucket"
val moviesPath = s"gs://$bucketName/ml-32m/movies.csv"
val hdfsPath = "hdfs:///user/shraman_jana/Q4/movies.csv"

// Step 3: Read the file from GCS
val data = spark.read
  .option("header", "true")  // Read the header from the CSV file
  .csv(moviesPath)

// Step 4: Write the file to HDFS with headers
data.write
  .option("header", "true")  // Include the header in the output
  .mode("overwrite")         // Overwrite if the file already exists
  .csv(hdfsPath)

println(s"File transferred from $moviesPath to $hdfsPath successfully!")

File transferred from gs://scala_assgn_bucket/ml-32m/movies.csv to hdfs:///user/shraman_jana/Q4/movies.csv successfully!


bucketName = scala_assgn_bucket
moviesPath = gs://scala_assgn_bucket/ml-32m/movies.csv
hdfsPath = hdfs:///user/shraman_jana/Q4/movies.csv
data = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [6]:
//Adding duplicates

// Step 1: Read the existing movies.csv from HDFS
val moviesPath = "hdfs:///user/shraman_jana/Q4/movies.csv"
val moviesDF = spark.read.option("header", "true").csv(moviesPath)

// Step 2: Add 1000 duplicates by appending the same DataFrame multiple times
val duplicateMoviesDF = moviesDF.limit(1000) // Take 1000 rows to duplicate
val moviesWithDuplicatesDF = moviesDF.union(duplicateMoviesDF) // Append duplicates

// Step 3: Write the updated DataFrame with duplicates back to the same HDFS path
moviesWithDuplicatesDF.write
  .option("header", "true")
  .mode("overwrite") // Overwrite the existing file
  .csv("hdfs:///user/shraman_jana/Q4/duplicated_movies.csv")

println("1000 duplicates inserted and file updated successfully!")

1000 duplicates inserted and file updated successfully!


moviesPath = hdfs:///user/shraman_jana/Q4/movies.csv
moviesDF = [movieId: string, title: string ... 1 more field]
duplicateMoviesDF = [movieId: string, title: string ... 1 more field]
moviesWithDuplicatesDF = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [10]:
// Step 2: Load movies.csv into a DataFrame from HDFS
val moviesPath = "hdfs:///user/shraman_jana/Q4/duplicated_movies.csv"
val moviesDF = spark.read.option("header", "true").csv(moviesPath)

// Step 3: Convert to RDD with composite key (movieId, title)
val moviesRDD = moviesDF.rdd.map(row => {
  val movieId = row.getString(row.fieldIndex("movieId"))
  val title = row.getString(row.fieldIndex("title"))
  val genres = row.getString(row.fieldIndex("genres"))
  ((movieId, title), genres) // Key: (movieId, title), Value: genres
})

// Combine genres for duplicate keys
val uniqueMoviesRDD = moviesRDD.reduceByKey((genres1, genres2) => s"$genres1|$genres2")

// Transform back to (movieId, title, genres) format
val cleanedMoviesRDD = uniqueMoviesRDD.map {
  case ((movieId, title), combinedGenres) => (movieId, title, combinedGenres)
}

val cleanedMoviesDF = cleanedMoviesRDD.toDF("movieId", "title", "genres")

// Step 4: Validation
val originalCount = moviesDF.count()
val deduplicatedCount = cleanedMoviesDF.count()
val duplicatesRemoved = originalCount - deduplicatedCount
println(s"Original record count: $originalCount")
println(s"Deduplicated record count: $deduplicatedCount")
println(s"Duplicates removed: $duplicatesRemoved")

Original record count: 88585
Deduplicated record count: 87585
Duplicates removed: 1000


moviesPath = hdfs:///user/shraman_jana/Q4/duplicated_movies.csv
moviesDF = [movieId: string, title: string ... 1 more field]
moviesRDD = MapPartitionsRDD[46] at map at <console>:56
uniqueMoviesRDD = ShuffledRDD[47] at reduceByKey at <console>:64
cleanedMoviesRDD = MapPartitionsRDD[48] at map at <console>:67
cleanedMoviesDF = [movieId: string, title: string ... 1 more field]
originalCount = 88585
deduplicatedCount = 87585
duplicatesRemoved = 1000


1000

In [12]:
// Step 6: Save the cleaned data as Avro files in GCP Cloud Storage
val outputPath = s"gs://$bucketName/Day16_17/Q4/cleaned_movies.avro"
cleanedMoviesDF.write
  .format("avro")
  .mode("overwrite")
  .save(outputPath)

println("Cleaned movies data saved successfully in Avro format.")

Cleaned movies data saved successfully in Avro format.


lastException = null
outputPath = gs://scala_assgn_bucket/Day16_17/Q4/cleaned_movies.avro


gs://scala_assgn_bucket/Day16_17/Q4/cleaned_movies.avro

In [17]:
//Method to count duplicates

def countDuplicates(inputDF: DataFrame): Long = {
    val groupedDF = inputDF
    .groupBy("movieId", "title")
    .count()

    val duplicateGroupsDF = groupedDF.filter(col("count") > 1)

    if (duplicateGroupsDF.head(1).isEmpty) {
        return 0L
    }

    val duplicateCount = duplicateGroupsDF
    .select(expr("sum(count - 1)")) 
    .collect()(0)(0)

    duplicateCount.toString.toLong
}

println("Initial movies duplicate count:")
println(countDuplicates(moviesDF))
println("Cleaned movies dataset duplicate count:")
println(countDuplicates(cleanedMoviesDF))

Initial movies duplicate count:
1000
Cleaned movies dataset duplicate count:


countDuplicates: (inputDF: org.apache.spark.sql.DataFrame)Long


0
